In [1]:
from tkinter import *
import pandas as pd 
import numpy as np

from sklearn.model_selection import StratifiedKFold,KFold,train_test_split
from sklearn.preprocessing import StandardScaler,Normalizer,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error


import tensorflow as tf
from tensorflow import keras
from  tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import EarlyStopping

tf.__version__

'2.4.1'

In [2]:

data = pd.DataFrame()
targetFeature = ""
r_model = Sequential(name="SignalPrediction_regression")
c_model = Sequential(name="SignalPrediction_classification")

# Function to update expression
# in the text entry box 
def press(filename,label): 
    print(filename.get())
    label.set(filename.get())
 
 
def importdata(filename,importDataprg):
    status ="Done"
    print("importdata",filename)
    try:
        global data
        data = pd.read_csv(filename)
        print("Done")
    except:
        status ="Error"
    
    importDataprg.set(status)
    
    
def loadtarget(target,prgStatus): 
    status ="Found"
    print(target)
    print(data.columns)    
    if target in data.columns:
        global targetFeature
        targetFeature = target
    else:     
        status ="Not Found"
    prgStatus.set(status)
    
def nn_model_r(input_dim,lr=0.01,momentum=0.9,dropout=0,kernel_initializer="normal"):
	# create model
  model = Sequential(name="SignalPrediction_regression")
  model.add(Dense(12,input_dim=input_dim, activation='relu',name="HL-1",kernel_initializer=kernel_initializer))
  model.add(Dropout(dropout,name ="DP-1"))
  model.add(Dense(6, activation='relu',name="HL-2",kernel_initializer=kernel_initializer))
  model.add(Dropout(dropout,name ="DP-2"))
  model.add(Dense(1,name="OL",activation="linear",kernel_initializer=kernel_initializer))
	# Compile model
  optimizer = SGD(lr=lr, momentum=momentum)
  model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=optimizer, metrics=['mae'])
  return model

def nn_model_c(input_dim,lr=0.001,kernel_initializer="he_uniform",l2Lambda=1):    
    model = Sequential(name="SignalPrediction_classification")
    model.add(Dense(11,input_dim=input_dim,activation='relu', name="HL-1",kernel_initializer=kernel_initializer,
                  kernel_regularizer=tf.keras.regularizers.l2(l2Lambda)))
    model.add(Dense(3,name="OL",activation="softmax",kernel_initializer=kernel_initializer))
    # Compile model
    optimizer = Adam(learning_rate=lr)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

def trainclassification(prgStatus):
    global data
    global targetFeature
    global c_model
    try:
        print("trainclassification")
        impTarget = targetFeature+"_Imp"
        data_copy = data.copy()
        data_copy[impTarget] = data_copy[targetFeature].apply(lambda x: np.where(x in [3,4,7,8], "Other", x))
        X = data_copy.drop([impTarget,targetFeature],axis=1)
        y = data_copy.filter([impTarget])
        print(X.head())
        print(y.head())
        label_encoder = LabelEncoder()
        label_encoder.fit(y.values.ravel())
        encoded_y = label_encoder.transform(y.values.ravel())        
        y = tf.keras.utils.to_categorical(encoded_y,num_classes=3)
        print(y)
        sc = StandardScaler()
        X = sc.fit_transform(X)
        c_model = nn_model_c(X.shape[1])
        print(c_model.summary())
        c_model.fit(X, y,  epochs=500,batch_size=16, verbose=1)        
        status ="Network Trained" 
    except:
        status ="Error" 
    prgStatus.set(status)
    
def save_c_model(prgStatus):
    status ="Saved Model to Disk"
    try:
        global c_model
        c_model.save("c_model_h5",save_format='h5')
    except:
        status ="Error" 
    prgStatus.set(status)     
    

def trainregression(prgStatus):
    status ="Network Training - In Progress" 
    global data
    global targetFeature
    global r_model
    try:
        print(targetFeature)
        X = data.drop(targetFeature,axis=1)
        y = data.filter([targetFeature])
        #X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=123)
        print(X.head())
        print(y.head())
        ##### Scaling the input
        scaling = StandardScaler()
        X = scaling.fit_transform(X)
        print(X)
        r_model = nn_model_r(X.shape[1])
        r_model.fit(X, y, epochs=200,batch_size=32, verbose=1)
        status ="Network Trained" 
    except:
        status ="Error"    
        
    prgStatus.set(status)
    
def save_r_model(prgStatus):
    status ="Saved Model to Disk"
    try:
        global r_model
        r_model.save("r_model_h5",save_format='h5')
    except:
        status ="Error" 
    prgStatus.set(status)    
 
# Driver code 
if __name__ == "__main__": 
    # create a GUI window 
    gui = Tk() 
    gui.configure(background="#d6d6c2")
    gui.title("Neural Network- Great Learning") 
    gui.geometry("680x300") 
    
    
    ### First Row ########
    filenamelabelStr = StringVar()
    
    filenamelabel = Label( gui, textvariable=filenamelabelStr,background="#d6d6c2",anchor="w")
    filenamelabelStr.set("Step 1: File Name")
    filenamelabel.grid(row=1, column=0,padx=20)
    
    
    filenameStr = StringVar()
    filenameStr.set("Part- 1,2&3 - Signal.csv")
    filenameValue = Entry(gui, textvariable=filenameStr) 
    filenameValue.grid(row=1, column=1,padx=20)
    
    importDataBtn = Button(gui, text='Import Data', fg='black', bg='#b3b3ff',padx="10", 
                    command=lambda:importdata(filenameStr.get(),importDataprg), height=1, width=15,cursor = "hand2") 
    importDataBtn.grid(row=1, column=2,padx=20) 
    
    importDataprg = StringVar()
    importDatalabel = Label( gui, textvariable=importDataprg,background="#d6d6c2" )
    #filenamelabelStr.set("Step 1: File Name")
    importDatalabel.grid(row=1, column=3,padx=20)
    
    
    ### Second Row ########
    
    targetlabelStr = StringVar()
    targetlabel = Label( gui, textvariable=targetlabelStr,background="#d6d6c2")
    targetlabelStr.set("Step 1: Target Column")
    targetlabel.grid(row=2, column=0,padx=20)
    
    
    targetStr = StringVar()    
    targetStr.set("Signal_Strength")
    targetValue = Entry(gui, textvariable=targetStr) 
    targetValue.grid(row=2, column=1,padx=20)
    
    importTargetBtn = Button(gui, text='Import Target', fg='black', bg='#b3b3ff',padx="10", 
                    command=lambda: loadtarget(targetStr.get(),importTargetprg), height=1, width=15,cursor = "hand2") 
    importTargetBtn.grid(row=2, column=2,padx=20) 
    
    importTargetprg = StringVar()
    importTargetlabel = Label( gui, textvariable=importTargetprg,background="#d6d6c2" )
    #filenamelabelStr.set("Step 1: File Name")
    importTargetlabel.grid(row=2, column=3,padx=20)
    
     ### 3rd Row ########
    
    nnrlabelStr = StringVar()
    nnrlabel = Label( gui, textvariable=nnrlabelStr,background="#d6d6c2",anchor="e")
    nnrlabelStr.set("Step 3: Neural Network regression")
    nnrlabel.grid(row=3, column=0,padx=20)
    
    
    ### 4th Row ########
    
    nnrtrainlabelStr = StringVar()
    nnrtrainlabel = Label( gui, textvariable=nnrtrainlabelStr,background="#d6d6c2")
    nnrtrainlabelStr.set("Regressor")
    nnrtrainlabel.grid(row=4, column=0,padx=20)
    
    
    
    nnrtrainBtn = Button(gui, text='Train', fg='black', bg='#b3b3ff',padx="10", 
                    command=lambda: trainregression(nnrtrainprg), height=1, width=15,cursor = "hand2") 
    nnrtrainBtn.grid(row=4, column=1,padx=20) 
    
    nnrtrainprg = StringVar()
    nnrtrainlabel = Label( gui, textvariable=nnrtrainprg,background="#d6d6c2" )
    #filenamelabelStr.set("Step 1: File Name")
    nnrtrainlabel.grid(row=4, column=2,padx=20)
    
     ### 5th Row ########
    
    nnrpickellabelStr = StringVar()
    nnrpickellabel = Label( gui, textvariable=nnrpickellabelStr,background="#d6d6c2")
    nnrpickellabelStr.set("Pickle")
    nnrpickellabel.grid(row=5, column=0,padx=20)
    
    
    
    nnrpickelBtn = Button(gui, text='Run', fg='black', bg='#b3b3ff',padx="10", 
                    command=lambda: save_r_model(nnrpickelprg), height=1, width=15,cursor = "hand2") 
    nnrpickelBtn.grid(row=5, column=1,padx=20) 
    
    nnrpickelprg = StringVar()
    nnrpickellabel = Label( gui, textvariable=nnrpickelprg,background="#d6d6c2" )
    #filenamelabelStr.set("Step 1: File Name")
    nnrpickellabel.grid(row=5, column=2,padx=20)
    
    
    
    
    ### 6th Row ########
    
    nnclabelStr = StringVar()
    nnclabel = Label( gui, textvariable=nnclabelStr,background="#d6d6c2",anchor="w")
    nnclabelStr.set("Step 4: Neural Network Classifier")
    nnclabel.grid(row=6, column=0,padx=20)
    
    
    ### 7th Row ########
    
    nnctrainlabelStr = StringVar()
    nnctrainlabel = Label( gui, textvariable=nnctrainlabelStr,background="#d6d6c2",anchor="e")
    nnctrainlabelStr.set("Classifier")
    nnctrainlabel.grid(row=7, column=0,padx=20)
    
    
    
    nnctrainBtn = Button(gui, text='Train', fg='black', bg='#b3b3ff',padx="10", 
                    command=lambda: trainclassification(nnctrainprg), height=1, width=15,cursor = "hand2") 
    nnctrainBtn.grid(row=7, column=1,padx=20) 
    
    nnctrainprg = StringVar()
    nnctrainlabel = Label( gui, textvariable=nnctrainprg,background="#d6d6c2" )
    #filenamelabelStr.set("Step 1: File Name")
    nnctrainlabel.grid(row=7, column=2,padx=20)
    
     ### 5th Row ########
    
    nncpickellabelStr = StringVar()
    nncpickellabel = Label( gui, textvariable=nncpickellabelStr,background="#d6d6c2")
    nncpickellabelStr.set("Pickle")
    nncpickellabel.grid(row=8, column=0,padx=20)
    
    
    
    nncpickelBtn = Button(gui, text='Run', fg='black', bg='#b3b3ff',padx="10", 
                    command=lambda: save_c_model(nncpickelprg), height=1, width=15,cursor = "hand2") 
    nncpickelBtn.grid(row=8, column=1,padx=20) 
    
    nncpickelprg = StringVar()
    nncpickellabel = Label( gui, textvariable=nncpickelprg,background="#d6d6c2" )
    #filenamelabelStr.set("Step 1: File Name")
    nncpickellabel.grid(row=8, column=2,padx=20)
   
    # start the GUI 
    gui.mainloop() 

importdata Part- 1,2&3 - Signal.csv
Done
Signal_Strength
Index(['Parameter 1', 'Parameter 2', 'Parameter 3', 'Parameter 4',
       'Parameter 5', 'Parameter 6', 'Parameter 7', 'Parameter 8',
       'Parameter 9', 'Parameter 10', 'Parameter 11', 'Signal_Strength'],
      dtype='object')
Signal_Strength
   Parameter 1  Parameter 2  Parameter 3  Parameter 4  Parameter 5  \
0          7.4         0.70         0.00          1.9        0.076   
1          7.8         0.88         0.00          2.6        0.098   
2          7.8         0.76         0.04          2.3        0.092   
3         11.2         0.28         0.56          1.9        0.075   
4          7.4         0.70         0.00          1.9        0.076   

   Parameter 6  Parameter 7  Parameter 8  Parameter 9  Parameter 10  \
0         11.0         34.0       0.9978         3.51          0.56   
1         25.0         67.0       0.9968         3.20          0.68   
2         15.0         54.0       0.9970         3.26          

In [3]:
# X = data.drop(targetFeature,axis=1)
# y = data.filter([targetFeature])
# #X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=123)
# print(X.head())
# print(y.head())
# ##### Scaling the input
# scaling = StandardScaler()
# X = scaling.fit_transform(X)
# X.shape

In [4]:
# print("trainclassification")
# impTarget = targetFeature+"_Imp"
# data[impTarget] = data[targetFeature].apply(lambda x: np.where(x in [3,4,7,8], "Other", x))
# X = data.drop([impTarget,targetFeature],axis=1)
# y = data.filter([impTarget])
# print(X.head())
# print(y.head())
# label_encoder = LabelEncoder()
# label_encoder.fit(y.values.ravel())
# encoded_y = label_encoder.transform(y.values.ravel())        
# y = tf.keras.utils.to_categorical(encoded_y,num_classes=3)
# print(y)
# sc = StandardScaler()
# X = sc.fit_transform(X)
# c_model = nn_model_c(X.shape[1])
# print(c_model.summary())
# c_model.fit(X, y,  epochs=500,batch_size=16, verbose=1)        
# status ="Network Trained"